# Reading the train and test files

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
combine=[train,test] 

In [ ]:
# Passenger ID can be dropped in train dataset but since we need it to prepare the submission file , we are keeping it in test.csv
train.drop(['PassengerId'],axis=1,inplace=True)
train.head()

### Checking for missing values

In [ ]:
print(train.info())

print(test.info())

### There are considerable number of missing values in Cabin in both test and train dataset. Hence dropping the same. 
There are missing values in Age, Embarkeded and Fare as well. But since the numbers are small, they can be managed. 

In [ ]:
for dataset in combine:
    dataset.drop(['Cabin'],axis=1,inplace=True)
    

In [ ]:
# Checking if Ticket numbers are unique
for data in combine:
    print('Dataset has ',data.shape[0],' entries and ',end=' ')
    print(data['Ticket'].nunique(),' values of Tickets')

In [ ]:
# Working with the missing values in Embarked

train[((train.Fare>78)&(train.Fare<83) )& (train['Pclass']==1)]

In [ ]:
# Replacing with the mode of the value of people in same class and similar fare. 
train['Embarked']=train['Embarked'].fillna('C')

# Does group and individual booking have an impact on survival rate?

In [ ]:
train[train['Ticket']=='W./C. 6607']

#### It seems that there are people who booked the ticket together, usually belonging to the same family and have same last name.

In [ ]:
for data in combine:
    group_bookings_tickets=data[data['Ticket'].duplicated()]['Ticket'].unique()
    data['group_booking']=[1 if ticket in group_bookings_tickets else 0 for ticket in data['Ticket']]

In [ ]:
train[['Survived','group_booking']].groupby('group_booking').mean()

#### Group booking has a higher rate of survival. 

In [ ]:
# Dropping the Ticket column as we hev extracted information from it

for data in combine:
    data.drop(['Ticket'],axis=1,inplace=True)

In [ ]:
train.head()

# Does gender of a person has an impact on his survival rate?


In [ ]:
## Label encoding the Sex column

for data in combine:
    data['Sex']=[1 if sex=='male' else 0 for sex in data['Sex']]

In [ ]:
# Checking the sruvival rate

train[['Sex','Survived']].groupby(['Sex']).mean()

#### Females have higher chance survival

# Does the Titles have an impact on Survival Rate?


In [ ]:
for data in combine:
    data['Salutation']=data['Name'].str.extract(' (\w+)\.')

In [ ]:
train[['Salutation','Survived']].groupby(['Salutation']).count().sort_values(by=['Survived'],ascending=False)

In [ ]:
for data in combine:
    data['Salutation']=data['Salutation'].replace(['Dr', 'Rev', 'Major', 'Col', 'Mlle','Mme', 'Ms', 'Capt', 'Lady', 'Jonkheer', 'Don', 'Countess', 'Sir','Dona'],'Others')
    # Converting groups with less than 10 count to a common group called others and label encoding others
    data['Title']=data['Salutation'].map({'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Others':4})
    data.drop(['Name'],axis=1,inplace=True)

In [ ]:
train[['Salutation','Survived']].groupby(['Salutation']).mean()

#### Passengers with Title Mrs had higher chances of surviving, followed by Miss and childred.


In [ ]:
for data in combine:
    data.drop(['Salutation'],axis=1,inplace=True) # deleting the column as we have already label encoded it. 
    

In [ ]:
train.head()

In [ ]:
#({'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Others':4})
for data in combine:
    import numpy as np
    Mr=np.zeros(data.shape[0])
    Miss=np.zeros(data.shape[0])
    Mrs=np.zeros(data.shape[0])
    Master=np.zeros(data.shape[0])
    for i,value in enumerate(data['Title']):
        try:
            if int(value)==0:
                Mr[i]=1
            elif int(value)==1:
                Miss[i]=1
            elif int(value)==2:
                Mrs[i]=1
            elif int(value)==3:
                Master[i]=1
        except:
            print(i,value)
    data['Mr']=Mr.astype(int)
    data['Mrs']=Mrs.astype(int)
    data['Miss']=Miss.astype(int)
    data['Master']=Master.astype(int)



In [ ]:
# Checking the dependence of Title on Survival rate
train[['Title','Survived']].groupby('Title').mean()

#### People with Title Miss and Mrs has higher survival rate followed by children. The least survival rate being that of Mr

In [ ]:
# Dropping the column Title
for data in combine:
    data.drop(['Title'],axis=1,inplace=True)

# Checking the dependence of Embarked Column on Survival Rate

In [ ]:
for data in combine:
    data['Embarked']=data['Embarked'].map({'S':0,'C':1,'Q':2})

In [ ]:
train[['Embarked','Survived']].groupby(['Embarked']).mean()

#### People who have started the Journey from Cherbourg has a higher survival rate compared to other boarding points

In [ ]:
train.head()

# Handling missing values in Age

In [ ]:
train[train['Age'].isna()]

In [ ]:
mr_mean=train[(train['Age'].notna()) & (train['Mr']==1)]['Age'].mean()
miss_mean=train[(train['Age'].notna()) & (train['Miss']==1)]['Age'].mean()
mrs_mean=train[(train['Age'].notna()) & (train['Mrs']==1)]['Age'].mean()
master_mean=train[(train['Age'].notna()) & (train['Master']==1)]['Age'].mean()
others_mean=train[(train['Age'].notna()) & (train['Master']+train['Mr']+train['Mrs']+train['Miss']==1)]['Age'].mean()

In [ ]:
for data in combine:
    for i,row in data.iterrows():
        if np.isnan(row['Age']):
            if row['Mr']==1:
                data.loc[i,'Age']=mr_mean
            elif row['Miss']==1:
                data.loc[i,'Age']=miss_mean
            elif row['Mrs']==1:
                data.loc[i,'Age']=mrs_mean
            elif row['Master']==1:
                data.loc[i,'Age']=master_mean
            else:
                data.loc[i,'Age']=others_mean

In [ ]:
train[train['Age'].isna()]

# Correcting the missing fare in test dataset

In [ ]:
test[test['Fare'].isna()]

In [ ]:
# Replacing the value of embarked in test dataset with the mean of value with same pclass and same embarked

test['Fare']=test['Fare'].fillna(train[(train['Pclass']==3) & (train['Embarked']==0)& (train['Mr']==1) &(train['group_booking']==0) & (train['Sex']==1) & ( train['Age']>60)]['Fare'].mean())

In [ ]:
test[test['Fare'].isna()]

# Checking for correlation between parameters

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
sns.heatmap(train.corr(),annot=True)

# Trying out different machine learning models

In [ ]:
class automl:
    def __init__(self,combine,target):
        self.xtrain=combine[0].drop([target],axis=1)
        self.ytrain=combine[0][target]
        self.test=combine[1]
        
        from sklearn.model_selection import train_test_split
        self.x_train,self.x_test,self.y_train,self.y_test=train_test_split(self.xtrain,self.ytrain,test_size=0.3,random_state=1)
        # Initialising the models to None. 
        # The models will be saved to these objects once the functions are called
        self.logitRegression=None
        self.decisionTree=None
        self.NaiveBayes=None
        self.randomForest=None
        self.MLP=None
        self.DNN=None
        self.output=None
        self.XGB=None
        
    def linear_regression(self):
        from sklearn.linear_model import LogisticRegression
        from sklearn.model_selection import KFold
        from sklearn.model_selection import cross_val_score
        lr=LogisticRegression(solver='liblinear')
        kf=KFold(n_splits=5)
        for tr,te in kf.split(self.xtrain):
            lr.fit(self.xtrain.iloc[tr],self.ytrain.iloc[tr])
            print(lr.score(self.xtrain.iloc[te],self.ytrain.iloc[te]))
        self.MLP=lr.fit(self.x_train,self.y_train)
        ret_model=lr
        self.logitRegression=lr.fit(self.xtrain,self.ytrain)
        return ret_model
        
    
    def mlp(self):
        from sklearn.neural_network import MLPClassifier
        from sklearn.model_selection import KFold
        model=MLPClassifier(hidden_layer_sizes=(256,128,64,32,16,8),random_state=1,max_iter=50,solver='adam')
        kf=KFold(n_splits=5)
        for tr,te in kf.split(self.xtrain):
            model.fit(self.xtrain.iloc[tr],self.ytrain.iloc[tr])
            print(model.score(self.xtrain.iloc[te],self.ytrain.iloc[te]))
        ret_model=model
        self.MLP=model.fit(self.xtrain,self.ytrain)
        return ret_model
        
    def predict(self,model):
        passenger=self.test['PassengerId']
        test=self.test.drop(['PassengerId'],axis=1)
        predictions=model.predict(test)
        out=pd.DataFrame()
        out['PassengerId']=passenger
        out['Survived']=predictions
        self.out=out
    
    def decision_tree(self):
        from sklearn.tree import DecisionTreeClassifier
        from sklearn.model_selection import KFold
        dt=DecisionTreeClassifier()
        kf=KFold(n_splits=5)
        for tr,te in kf.split(self.xtrain):
            dt.fit(self.xtrain.iloc[tr],self.ytrain.iloc[tr])
            print(dt.score(self.xtrain.iloc[te],self.ytrain.iloc[te]))
        ret_model=dt
        self.decisionTree=dt
        return ret_model
    
    def random_forest(self):
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.model_selection import KFold
        rf=RandomForestClassifier()
        kf=KFold(n_splits=5)
        for tr,te in kf.split(self.xtrain):
            rf.fit(self.xtrain.iloc[tr],self.ytrain.iloc[tr])
            print(rf.score(self.xtrain.iloc[te],self.ytrain.iloc[te]))
        ret_model=rf
        self.randomForest=rf
        return ret_model
    
    def xgboost(self):
        from xgboost import XGBClassifier
        from sklearn.model_selection import KFold
        xgb=XGBClassifier()
        kf=KFold(n_splits=5)
        for tr,te in kf.split(self.xtrain):
            xgb.fit(self.xtrain.iloc[tr],self.ytrain.iloc[tr])
            print(xgb.score(self.xtrain.iloc[te],self.ytrain.iloc[te]))
        ret_model=xgb
        self.XGB=xgb.fit(self.xtrain,self.ytrain)
        return ret_model

    def naive_bayes(self):
        from sklearn.naive_bayes import GaussianNB
        from sklearn.model_selection import KFold
        gnb=GaussianNB()
        kf=KFold(n_splits=5)
        for tr,te in kf.split(self.xtrain):
            gnb.fit(self.xtrain.iloc[tr],self.ytrain.iloc[tr])
            print(gnb.score(self.xtrain.iloc[te],self.ytrain.iloc[te]))
        ret_model=gnb
        self.XGB=gnb.fit(self.xtrain,self.ytrain)
        return ret_model
    def save_out(self,filename):
        self.out.to_csv(filename,index=False)
        


In [ ]:
auto=automl(combine,'Survived')

In [ ]:
model=auto.mlp()
auto.predict(model)
auto.save_out('MLP.csv')

In [ ]:
model=auto.linear_regression()
auto.predict(model)
auto.save_out('LR1.csv')

In [ ]:
model=auto.decision_tree()
auto.predict(model)
auto.save_out('DT.csv')

In [ ]:
model=auto.random_forest()
auto.predict(model)
auto.save_out('RF.csv')

In [ ]:
model=auto.xgboost()
auto.predict(model)
auto.save_out('XGB.csv')

In [ ]:
model=auto.xgboost()
auto.predict(auto.XGB)
auto.save_out('XGB1.csv')

In [ ]:
model=auto.naive_bayes()
auto.predict(model)
# auto.save_out('XGB1.csv')

In [ ]:
one=pd.read_csv('./MLP.csv')
two=pd.read_csv('./LR.csv')

In [ ]:
one.head(10)

In [ ]:
two.head(10)

In [ ]:
import pandas as pd
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
combine=[train,test] 

In [ ]:
# Passenger ID can be dropped in train dataset but since we need it to prepare the submission file , we are keeping it in test.csv
train.drop(['PassengerId'],axis=1,inplace=True)
train.head()

### Checking for missing values

In [ ]:
print(train.info())

print(test.info())

In [ ]:
train[train['Embarked'].isna()]

In [ ]:
# The two people have booked the ticket together since they have the same ticket number. 
train[train['Ticket']=='113572']

In [ ]:
# Let us look at people with same fare and same Pclass to identify the Embarked location
train[(train['Pclass']==1) & ((train.Fare>78) & (train.Fare<83))]

### There are considerable number of missing values in Cabin in both test and train dataset. Hence dropping the same. 
There are missing values in Age, Embarkeded and Fare as well. But since the numbers are small, they can be managed. 

In [ ]:
for dataset in combine:
    dataset.drop(['Cabin'],axis=1,inplace=True)
    

In [ ]:
# Checking if Ticket numbers are unique
for data in combine:
    print('Dataset has ',data.shape[0],' entries and ',end=' ')
    print(data['Ticket'].nunique(),' values of Tickets')

# Does group and individual booking have an impact on survival rate?

In [ ]:
train[train['Ticket']=='W./C. 6607']

#### It seems that there are people who booked the ticket together, usually belonging to the same family and have same last name.

In [ ]:
for data in combine:
    group_bookings_tickets=data[data['Ticket'].duplicated()]['Ticket'].unique()
    data['group_booking']=[1 if ticket in group_bookings_tickets else 0 for ticket in data['Ticket']]

In [ ]:
train[['Survived','group_booking']].groupby('group_booking').mean()

#### Group booking has a higher rate of survival. 

In [ ]:
# Dropping the Ticket column as we hev extracted information from it

for data in combine:
    data.drop(['Ticket'],axis=1,inplace=True)

In [ ]:
train.head()

# Does gender of a person has an impact on his survival rate?


In [ ]:
## Label encoding the Sex column

for data in combine:
    data['Sex']=[1 if sex=='male' else 0 for sex in data['Sex']]

In [ ]:
# Checking the sruvival rate

train[['Sex','Survived']].groupby(['Sex']).mean()

#### Females have higher chance survival

# Does the Titles have an impact on Survival Rate?


In [ ]:
for data in combine:
    data['Salutation']=data['Name'].str.extract(' (\w+)\.')

In [ ]:
train[['Salutation','Survived']].groupby(['Salutation']).count().sort_values(by=['Survived'],ascending=False)

In [ ]:
for data in combine:
    data['Salutation']=data['Salutation'].replace(['Dr', 'Rev', 'Major', 'Col', 'Mlle','Mme', 'Ms', 'Capt', 'Lady', 'Jonkheer', 'Don', 'Countess', 'Sir','Dona'],'Others')
    # Converting groups with less than 10 count to a common group called others and label encoding others
    data['Title']=data['Salutation'].map({'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Others':4})
    data.drop(['Name'],axis=1,inplace=True)

In [ ]:
train[['Salutation','Survived']].groupby(['Salutation']).mean()

#### Passengers with Title Mrs had higher chances of surviving, followed by Miss and childred.


In [ ]:
for data in combine:
    data.drop(['Salutation'],axis=1,inplace=True) # deleting the column as we have already label encoded it. 
    

In [ ]:
train.head()

In [ ]:
#({'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Others':4})
for data in combine:
    import numpy as np
    Mr=np.zeros(data.shape[0])
    Miss=np.zeros(data.shape[0])
    Mrs=np.zeros(data.shape[0])
    Master=np.zeros(data.shape[0])
    for i,value in enumerate(data['Title']):
        try:
            if value==0:
                Mr[i]=1
            elif value==1:
                Miss[i]=1
            elif value==2:
                Mrs[i]=1
            elif value==3:
                Master[i]=1
        except:
            print(i,value)
    data['Mr']=Mr.astype(int)
    data['Mrs']=Mrs.astype(int)
    data['Miss']=Miss.astype(int)
    data['Master']=Master.astype(int)

In [ ]:
train.head(5)


In [ ]:
for data in combine:
    data.drop(['Title'],axis=1,inplace=True)

### Checking the influence of Embarked column on Survival rate

In [ ]:
train.Embarked.isna().sum()

In [ ]:
# Embarked has two Nan values. Let us try to fill them with value from similar data from the dataset
train[train.Embarked.isna()]